# k-Means Solutions

<hr style="clear:both">

**Author:** Sabri El Amrani

<hr style="clear:both">

In [ ]:
# Function to align all tables to the left (useful for later on)

In [ ]:
%%html
<style>
table {float:left}
</style>

### Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from typing import Any, Callable
import pickle

from sklearn.model_selection import KFold
from sklearn.metrics import silhouette_score

# Helper file with functions for pre-processing and visualization
import helpers

## 0. Intro

Let's continue last week's notebook on the identification of species of plants using reflectance spectra. Last week, we used PCA to help reduce the dimensionality of these spectra. Let's now implement k-means to find out how these datapoints naturally cluster.

## 1. Data loading

Let's start by loading the data we prepared last time.

In [ ]:
# Unpickle arrays arrays created in the pca notebook, similar to PCA, we use all data for training
with open('data/pca_preprocessed_angers_dataset.pkl', 'rb') as f:
    X_train, y_train, label_map = pickle.load(f)

Run the following cells to remember what our data looks like.

In [ ]:
print(f"Map from label to label name: {label_map}")

In [ ]:
# Show shapes
print('Training set shape:')
print(f'X: {X_train.shape}, y: {y_train.shape}')

Let's start by keeping the first two principal components for the sake of this exercice. We will see whether they suffice to properly cluster our data.

In [ ]:
X_train = X_train[:,:2]

print('Training set shape:')
print(f'X: {X_train.shape}, y: {y_train.shape}')

## 2. Data visualization & scaling

As a first step, let's scale our data as we often do in machine learning.

In [ ]:
### START CODE HERE ###
# Compute the mean and standard deviation for each feature of the training set
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
### END CODE HERE ###


# Implement the normalize function
def normalize(X: np.ndarray, mean: np.ndarray, std: np.ndarray):
    """ Normalization of array using Z-score standardization
     Args:
        X: Dataset of shape (N, D)
        mean: Mean of shape (D, )
        std: Standard deviation of shape(D, )
    """
    ### START CODE HERE ### (≈ 1 line of code)
    X_normalized = (X - mean) / std
    ### END CODE HERE ###
    return X_normalized

# Normalize features of the training, val and test set using the mean and std of the training set features
X_train = normalize(X_train, mean, std)

# Let's rename the features to indicate that they've been normalized
feature_names = ['X', 'Y']

Let's take a look at our data using the `plot_labeled()` and `plot_unlabeled()` functions of `helpers.py`.

In [ ]:
# Plot normalized training and testing data

### START CODE HERE ###
helpers.plot_labeled(X_train, y_train, label_map, feature_names, title="Training data")
### END CODE HERE ###

Does our data naturally form clusters? Do these clusters match the underlying species? Hard to say, so let's use k-means to answer these questions.

## 3. K-Means

### Reminder of the algorithm:

1. __Initialization:__ Start by randomly selecting K points from the dataset. These points will act as the initial cluster centers.
2. __Assignment:__ For each data point in the dataset, calculate the distance between that point and each of the K centers. Assign the data point to the cluster whose center is closest to it. This step effectively forms K clusters.
3. __Update centers:__ Once all data points have been assigned to clusters, recalculate the centers of the clusters by taking the mean of all data points assigned to each cluster.
4. __Repeat:__ Repeat steps 2 and 3 until convergence. Convergence occurs when the centers no longer change significantly or when a specified number of iterations is reached.
5. __Final Result:__ Once convergence is achieved, the algorithm outputs the final cluster centers and the assignment of each data point to a cluster.

[Source](https://www.analyticsvidhya.com/blog/2019/08/comprehensive-guide-k-means-clustering/)

Let's implement the various steps one by one.

### 3.1 Initialization 

In [ ]:
def initialize_centers(data, k):
    """
    Selects k random points from the dataset as initial cluster centers.

    Parameters:
    data (ndarray): The dataset from which to select initial centers.
    k (int): The number of centers to initialize.

    Returns:
    ndarray: An array of k initial centers randomly chosen from the dataset.
    """
    ### START CODE HERE ###
    indices = np.random.choice(data.shape[0], k, replace=False)
    return data[indices]
    ### END CODE HERE ###

### 3.2 Assign data points to nearest center

Implement the euclidean distance.

__Hint:__ Don't forget to use broadcasting.

In [ ]:
def euclidean_dist(data, centers):
    """
    Computes the Euclidean distance between each data point and each center.

    Parameters:
    data (ndarray): The dataset of shape (n_samples, n_features).
    centers (ndarray): The centers of shape (k, n_features).

    Returns:
    ndarray: A distance matrix of shape (k, n_samples) where each entry (i, j)
             is the distance between the ith center and the jth data point.
    """
    ### START CODE HERE ###
    distances = np.sqrt(((data - centers[:, np.newaxis])**2).sum(axis=2))
    return distances
    ### END CODE HERE ###

Now assign each datapoint to the nearest center.

In [ ]:
def assign_clusters(data, centers, distance_fn):
    """
    Assigns each data point to the nearest center.

    Parameters:
    data (ndarray): The dataset of shape (n_samples, n_features).
    centers (ndarray): The centers of shape (k, n_features).
    distance_fn (function): A function to compute distances between data points and centers.

    Returns:
    ndarray: An array of shape (n_samples,) where each value is the index of the nearest center.
    """
    ### START CODE HERE ###
    distances = distance_fn(data, centers)
    return np.argmin(distances, axis=0)
    ### END CODE HERE ###

### 3.3 Update centers

Updated center: mean of the assigned data points.

In [ ]:
def update_centers(data, labels, k):
    """
    Updates center positions by calculating the mean of data points assigned to each center.

    Parameters:
    data (ndarray): The dataset of shape (n_samples, n_features).
    labels (ndarray): An array of shape (n_samples,) containing cluster assignments.
    k (int): The number of centerss or clusters.

    Returns:
    ndarray: An array of shape (k, n_features) with the updated center positions.
    """
    ### START CODE HERE ###
    new_centers = np.array([data[labels == i].mean(axis=0) for i in range(k)])
    return new_centers
    ### END CODE HERE ###

### 3.4 Check convergence

Verify that the centers have not moved significantly since the previous iteration. In other words, ensuring that no center has changed by more than a specified minimum value, referred to as the tolerance (a hyperparameter set to 1e-5 in this case).

Hint: Use np.all() to check whether all elements in an array are less than the tolerance.

In [ ]:
def has_converged(old_centers, new_centers, tolerance=1e-5):
    """
    Checks if the center positions have converged.

    Parameters:
    old_centers (ndarray): The previous center positions.
    new_centers (ndarray): The updated center positions.
    tolerance (float, optional): The convergence threshold. Default is 1e-5.

    Returns:
    bool: True if all center changes are within the tolerance, otherwise False.
    """
    ### START CODE HERE ###
    return np.all(np.abs(old_centers- new_centers) < tolerance)
    ### END CODE HERE ###

### 3.5 K-Means algorithm: everything combined

Don't forget to stop the algorithm if either of these two conditions is met:
1. The algorithm has converged.
2. The maximum number of iterations has been reached.

In [ ]:
def kmeans(data, k, distance_fn, max_iters=100, tolerance=1e-5):
    """
    Runs the K-Means clustering algorithm.

    Parameters:
    data (ndarray): The dataset of shape (n_samples, n_features).
    k (int): The number of clusters.
    distance_fn (function): A function to compute distances between data points and centers.
    max_iters (int, optional): The maximum number of iterations to run the algorithm. Default is 100.
    tolerance (float, optional): The convergence threshold. Default is 1e-5.

    Returns:
    tuple: A tuple containing:
        - ndarray: The final center positions of shape (k, n_features).
        - ndarray: The cluster labels for each data point, of shape (n_samples,).
    """
    np.random.seed(1)

    ### START CODE HERE ###
    # Step 1: Initialize centers
    centers = initialize_centers(data, k)
    
    for _ in range(max_iters):
        # Step 2: Assign clusters
        labels = assign_clusters(data, centers, distance_fn)
        
        # Step 3: Update centers
        new_centers = update_centers(data, labels, k)
        
        # Step 4: Check for convergence
        if has_converged(centers, new_centers, tolerance):
            break
        
        centers = new_centers
    ### END CODE HERE ###

    return centers, labels

Now let's apply our algorithm the the our reduced Angers dataset.

In [ ]:
# Define the number of clusters, try 2, 3 and 4
k = 3

### START CODE HERE ###
# Run K-Means algorithm
centers, labels = kmeans(X_train, k, euclidean_dist)
### END CODE HERE ###

In [ ]:
label_map = {i: f'Cluster {i}' for i in range(k)}

helpers.plot_labeled(X_train, labels, label_map, feature_names, title="Training data - clusters")

**Note:** the clustering performance depends heavily on the initialization of the cluster centers. You can play with the random seed at the beginning of the kmeans(.) function. Check how clustering performs with different initializations.

## 4. Selecting hyperparameters (bonus reading)

To choose the number of clusters, we can consider using the two following methods [(Source)](https://medium.com/@zalarushirajsinh07/the-elbow-method-finding-the-optimal-number-of-clusters-d297f5aeb189):
1. Elbow method: we are looking for an inflection point in the plot of the sum of squared distances, as shown in the picture below.
2. Silhouette score: the optimal k should correspond to the maximum of the silhouette plot.

<img src="images/elbow.png" style="width:500px"/>

Combine the insights of these two plots to determine the optimal number of clusters k.
Given the small size of the dataset, we didn't split it into training/validation/test sets. Therefore, we will use the [cross-validation](https://scikit-learn.org/stable/modules/cross_validation.html) (cf. logistic regression notebook) method on the training set to find the optimal k.

That concludes this tutorial on k-means. Thank you for taking part!